In [2]:
import sys

sys.path.append("../../../")

from src.config import *
from src.taxonomy.head.headParsing import find_head

In [2]:
find_head("CommonsRoot")

2022-05-23 19:48:10 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-05-23 19:48:10 INFO: Use device: gpu
2022-05-23 19:48:10 INFO: Loading: tokenize
2022-05-23 19:48:12 INFO: Loading: pos
2022-05-23 19:48:12 INFO: Loading: constituency
2022-05-23 19:48:13 INFO: Done loading processors!


['Commonsroot']

In [3]:
tests = [
    "Films shot in New Jersey",
    "American comedy films",
    "1978 debut albums",
    "Allegories of humility",
    "1983 in water transport in Japan",
    "Civil parishes in North Yorkshire",
    "The New Orleans Bee October 1908",
    "Socata TB-10 at Flugplatz Uetersen",
    "Portrait paintings by Alexei Harlamov",
    "Family portraits of Russia",
    "Maria Pavlovna Abamelik-Lazareva (Demidova)",
    "Historical photographs of Kano",
    "Maps of Vigneux-sur-Seine",
    "California Digital Library",
    "Old books from American Libraries",
    "Schlosspark Vösendorf",
    "Comedy films of the United States",
]


for test in tests:
    print(test, "\t- head: " + find_head(test)[0])

Films shot in New Jersey 	- head: Films
American comedy films 	- head: Comedy films
1978 debut albums 	- head: Debut albums
Allegories of humility 	- head: Allegories
1983 in water transport in Japan 	- head: 1983
Civil parishes in North Yorkshire 	- head: Parishes
The New Orleans Bee October 1908 	- head: New orleans bee october
Socata TB-10 at Flugplatz Uetersen 	- head: Socata tb
Portrait paintings by Alexei Harlamov 	- head: Portrait paintings
Family portraits of Russia 	- head: Family portraits
Maria Pavlovna Abamelik-Lazareva (Demidova) 	- head: Lazareva
Historical photographs of Kano 	- head: Photographs
Maps of Vigneux-sur-Seine 	- head: Maps
California Digital Library 	- head: Library
Old books from American Libraries 	- head: Books
Schlosspark Vösendorf 	- head: Schlosspark vösendorf
Comedy films of the United States 	- head: Comedy films


In [5]:
import stanza

nlp = stanza.Pipeline(lang="en", processors="tokenize,pos,constituency")

2022-05-12 13:43:07 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-05-12 13:43:07 INFO: Use device: gpu
2022-05-12 13:43:07 INFO: Loading: tokenize
2022-05-12 13:43:07 INFO: Loading: pos
2022-05-12 13:43:07 INFO: Loading: constituency
2022-05-12 13:43:08 INFO: Done loading processors!


In [14]:
nlp("Comedy films of the United States").sentences[0].constituency

(ROOT (NP (NP (NN Comedy) (NNS films)) (PP (IN of) (NP (DT the) (NNP United) (NNPS States)))))

## Profiling

In [4]:
import time

import numpy as np
import pandas as pd

In [5]:
files = pd.read_parquet(FILES_PATH)

In [6]:
cats = files.categories[:300].explode().dropna().values
cats.shape

(838,)

In [7]:
start_time = time.time()

for i, category in enumerate(cats):
    head = find_head([category])

print("--- %s seconds ---" % (time.time() - start_time))

--- 74.00974297523499 seconds ---


In [8]:
start_time = time.time()

heads = list(find_head(cats))

print("--- %s seconds ---" % (time.time() - start_time))

--- 5.9921183586120605 seconds ---


In [9]:
list(find_head(cats))

['Department',
 'Bequest',
 '18th - century art',
 'Women',
 'Ship',
 'Yard',
 'Felice cerruti bauduc',
 'Victor emmanuel',
 'Engravings',
 'Engravings',
 'Engravings',
 'Archivio libreria antiquaria bourlot',
 'Pedro lira',
 'People',
 'Works',
 'Paintings',
 'Pottery',
 'Food and drug administration',
 'Circle',
 'Blue',
 'Tobacco',
 'Art',
 'Harmon foundation',
 'Selma burke',
 'State border signs',
 'Road signs',
 'Roads',
 'Porcelain',
 'Nastaliq',
 'Language',
 'Battle',
 'Engravings',
 'People',
 'Art journal',
 'William brassey hole',
 'Text',
 'Pro-suffrage cartoons',
 'Kenneth r. chamberlain',
 'Puck',
 'Cartoons',
 'Gmc sierra',
 'Gmc denali',
 'Chevrolet captiva sport',
 'De-valleyfield',
 'Pontiac grand prix',
 'Burlington breakwater',
 'Boathouses',
 'Stromatolites',
 'Jenolan caves',
 'Switzerland photographs',
 'Photographs',
 'Trees',
 'Messikomer eich',
 'Sesson shūkei',
 'Paintings',
 'Paintings',
 'Doves',
 'Mynah birds',
 '',
 'People',
 'Boats',
 'Makoko',
 'Niger

In [10]:
s = 80
cats[s]

"St Augustine's Catholic Church, Balmain"

In [11]:
find_head.nlp(".\n\n".join(cats)).sentences[s].constituency

(ROOT (NP (NP (NP (NNP St) (NNP Augustine) (POS 's)) (NNP Catholic) (NNP Church)) (, ,) (NP (NNP Balmain)) (. .)))

In [13]:
find_head.nlp(cats[s] + ".").sentences[0].constituency

(ROOT (NP (NP (NP (NNP St) (NNP Augustine) (POS 's)) (NNP Catholic) (NNP Church)) (, ,) (NP (NNP Balmain)) (. .)))

In [14]:
# batched = find_head.nlp('.\n\n'.join(cats))
# same = np.zeros(len(cats))
# for s in range(len(cats)):
#     same[s] = find_head.nlp(cats[s] + '.').sentences[0].constituency == batched.sentences[s].constituency